In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!pip install konlpy
!pip install flask-ngrok

  Using cached urllib3-1.26.7-py2.py3-none-any.whl (138 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.down

In [3]:
#crawling
import heapq
from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time

#nlp
from konlpy.tag import Okt
from collections import Counter

#server 
from flask import Flask, url_for, redirect, render_template, request
from flask_ngrok import run_with_ngrok
import os

id_ = 'your id'
password = 'your password'

import pandas as pd

df = pd.DataFrame(columns = ['text'])

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("user-agent={Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36}")
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

url = 'https://everytime.kr/login'

driver.get(url)

driver.find_element_by_name("userid").send_keys(id_)

driver.find_element_by_name("password").send_keys(password)

driver.find_element_by_tag_name("input").send_keys(Keys.RETURN)

time.sleep(2)
driver.find_element_by_xpath('//*[@id="submenu"]/div[1]/div[2]/ul/li[1]//a').click()

time.sleep(0.5)

res = driver.page_source
soup = BeautifulSoup(res,"html.parser")
data_name = soup.select('#container > div.wrap.articles > article > a > h2')
data_text = soup.select('#container > div.wrap.articles > article > a > p')

for name,text in zip(data_name,data_text):
    df.loc[df.shape[0]] = [name.get_text() + ' ' + text.get_text()]  
 
time.sleep(1)
def next_page():
    driver.find_element_by_css_selector('#container > div.wrap.articles > div.pagination > a.next').click()
    time.sleep(0.5)
    
    res = driver.page_source
    soup = BeautifulSoup(res,"html.parser")
    data_name = soup.select('#container > div.wrap.articles > article > a > h2')
    data_text = soup.select('#container > div.wrap.articles > article > a > p')   
    for name,text in zip(data_name,data_text):
        df.loc[df.shape[0]] = [name.get_text() + ' ' + text.get_text()]      
time.sleep(0.5)
for page_roof in range(50):
    next_page()
driver.quit()

count_dic={}

#text data processing and heapify

for text in df.text:
  twitter = Okt()
  noun = twitter.nouns(str(text))
  count_dic.update(Counter(noun))

noun_list = Counter(count_dic)
noun_list = list(noun_list.items())

heapq.heapify(noun_list)

# insert into max heap
data = []
max_heap = []
for noun in noun_list:
  heapq.heappush(max_heap, (-noun[1], noun[1],noun[0]))

for i in range(10):
  trender = heapq.heappop(max_heap)
  data.append(trender[2])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: use options instead of chrome_options
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


In [5]:
# server

TEMPLATE_FOLDER = '/content/drive/MyDrive/algo/templates' # 임의의 경로
STATIC_FOLDER = '/content/drive/MyDrive/algo/static' # 임의의 경로
app = Flask(__name__, template_folder=TEMPLATE_FOLDER, static_folder = STATIC_FOLDER)
run_with_ngrok(app)   

@app.route('/')
def index():
    return render_template('index.html',data = {1:data[0], 2:data[1], 3:data[2], 4:data[3], 5:data[4], 6:data[5], 7:data[6], 8:data[7], 9:data[8], 10:data[9]})

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://3f31-35-245-155-57.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [25/Nov/2021 12:36:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2021 12:36:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2021 12:36:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2021 12:36:47] "GET /static/css/index.css HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2021 12:36:47] "GET /static/css/index.css HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2021 12:36:48] "GET /favicon.ico HTTP/1.1" 404 -
